In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import scipy.stats as stats

In [ ]:
def convert_mg_to_mmolar(conc_mg_ml, mass):
    return round(conc_mg_ml/mass*1000, ndigits=3)

convert_mg_to_mmolar(0.225, 129.1)

## Show OD at 237 varies linearly with product conentration

In [ ]:
substrate_product_dosing = pd.read_excel('./spectra/ratio_5FC_5FU_reading_1.xlsx', skiprows=[0]+[x for x in range(82,107)],
                                 na_values='Overflow')
substrate_product_dosing = pd.melt(substrate_product_dosing, id_vars='[nm]', var_name='wavelength', value_name='OD')
substrate_product_dosing['wavelength'] = substrate_product_dosing['wavelength'].astype(int)
substrate_product_dosing['OD'] = substrate_product_dosing['OD'].astype(float)
substrate_product_dosing.rename(columns = {'[nm]':'row'}, inplace=True)
substrate_product_dosing['rep']=1
substrate_product_dosing

In [ ]:
for rep in [2,3,4,5]:
    data_df = pd.read_excel('./spectra/ratio_5FC_5FU_reading_'+str(rep)+'.xlsx', skiprows=[0]+[x for x in range(82,107)],
                                 na_values='Overflow')
    data_df = pd.melt(data_df, id_vars='[nm]', var_name='wavelength', value_name='OD')
    data_df['wavelength'] = data_df['wavelength'].astype(int)
    data_df['OD'] = data_df['OD'].astype(float)
    data_df.rename(columns = {'[nm]':'row'}, inplace=True)
    data_df['rep']=rep

    
    substrate_product_dosing=substrate_product_dosing.append(data_df, ignore_index=True)
    
substrate_product_dosing

In [ ]:
FU_frac_dict =     {'3':0,
                    '4':1,
                    '5':0.5,
                    '6':0.75,
                    '7':0.25,
                    '8':0.9,
                    '9':0.1,
                    '10':0.95,
                    '11':0.05,
                    '12':0.376}

sustrate_start_dict =  {'A':convert_mg_to_mmolar(0.225/2, 129.1),
                        'B':convert_mg_to_mmolar(0.2/2, 129.1),
                        'C':convert_mg_to_mmolar(0.175/2, 129.1),
                        'D':convert_mg_to_mmolar(0.15/2, 129.1),
                        'E':convert_mg_to_mmolar(0.125/2, 129.1),
                        'F':convert_mg_to_mmolar(0.1/2, 129.1),
                        'G':convert_mg_to_mmolar(0.075/2, 129.1),
                        'H':convert_mg_to_mmolar(0/2, 129.1)}

substrate_product_dosing['FC+FU_conc'] = substrate_product_dosing.apply(lambda x: sustrate_start_dict[x.row[0]], axis=1)
substrate_product_dosing['FU_frac'] = substrate_product_dosing.apply(lambda x: FU_frac_dict[x.row[1:]], axis=1)
substrate_product_dosing['FU_conc'] = substrate_product_dosing['FC+FU_conc'] * substrate_product_dosing['FU_frac']

substrate_product_dosing

In [ ]:
test_775 = substrate_product_dosing[substrate_product_dosing['FC+FU_conc']==0.775]

test_775

In [ ]:
plt.figure(figsize=(16,12))

sns.lineplot(data=test_775, x='wavelength', y='OD', hue='FU_frac', legend='full', palette='cividis')
plt.xlim(200,350)
plt.legend(fontsize=12, title='Fraction of 5-FC->5-FU')
plt.xlabel('Wavelength (nm)', fontsize=14)
plt.ylabel('Absorbance (OD)', fontsize=14)


plt.axvline(237, color='k', linestyle='--')



plt.savefig('./figures/spectra_775uM.svg', format='svg', dpi=300)

In [ ]:
test_775_237nm = test_775[test_775['wavelength']==237]

sns.regplot(data = test_775_237nm, x='FU_frac', y='OD', color='k', robust=True, line_kws={'color':'blue', 'linestyle':'--', 'alpha':0.5}, x_jitter=.01, scatter_kws={'alpha':0.25})



In [ ]:
all_237 = substrate_product_dosing[substrate_product_dosing['wavelength']==237]
all_237

In [ ]:
sns.lmplot(x="FU_frac", y="OD", hue="FC+FU_conc", data=all_237, palette='cividis')

plt.title('237 nm')
plt.ylabel('Absorbance (OD)', fontsize=14)
plt.xlabel('Fraction of 5-FC->5-FU', fontsize=14)

In [ ]:
sns.lmplot(x="FU_conc", y="OD", hue="FC+FU_conc", data=all_237, palette='cividis')

plt.title('237 nm')
plt.ylabel('Absorbance (OD)', fontsize=14)
plt.xlabel('5-FU [mM]', fontsize=14)

In [ ]:
all_237['FC+FU_conc'].unique()

In [ ]:
conc_regress_dict = {}

for conc in [0.871, 0.775, 0.678, 0.581, 0.484, 0.387, 0.29 , 0.]:
    print(stats.linregress(all_237[all_237['FC+FU_conc']==conc]['OD'], all_237[all_237['FC+FU_conc']==conc]['FU_conc']))
    conc_regress_dict[conc] = stats.linregress(all_237[all_237['FC+FU_conc']==conc]['OD'], all_237[all_237['FC+FU_conc']==conc]['FU_conc'])



## Stability data

In [ ]:
ti_df = pd.read_excel('./data/ti_heteromers.xlsx', index_col=0)
ti_df

ti_run_1 = ti_df[ti_df['run']==1]

In [ ]:
ti_df

In [ ]:
ti_df[ti_df['sample'].isin(['WT_1', 'E64V', 'R73G', 'M100W', 'E64V-R73G', 'E64V-M100W', 'D155S', 'D155S-M100W'])]

In [ ]:
plt.figure(figsize=(8,8), facecolor='white')

ax = sns.swarmplot(data=ti_df[ti_df['sample'].isin(['WT_1', 'E64V', 'R73G', 'M100W', 'E64V-R73G', 'E64V-M100W', 'WT_2','D155S', 'D155S-M100W'])], 
                   order = ['WT_1', 'E64V', 'R73G', 'M100W', 'E64V-R73G', 'E64V-M100W', 'WT_2','D155S', 'D155S-M100W'],
                   y='sample', x='Ti', dodge=True, size=10, palette=['black','#C03830','black','black','#489FA7','#489FA7', 'black','#C03830'], orient='h')

sample_list = ['WT_1', 'E64V', 'R73G', 'M100W', 'E64V-R73G', 'E64V-M100W', 'D155S', 'D155S-M100W']

for x in range(0,6):
    mean = np.mean(ti_run_1[ti_run_1['sample']==sample_list[x]]['Ti'])
    sem = stats.sem(ti_run_1[ti_run_1['sample']==sample_list[x]]['Ti'])
    interval = stats.t.interval(alpha=0.95, df=3-1, loc=mean, scale=sem)
    #plt.plot([x,x], interval, '-', lw=3, zorder=9, color='grey')
    print(stats.t.interval(alpha=0.95, df=3-1, loc=mean, scale=sem))
stats.t.interval(alpha=0.95, df=3-1, loc=37.98, scale=0.05)


plt.xlabel('T$_{m}$  ($^\circ$C)', fontsize=22)
plt.xticks(fontsize=18)

plt.yticks([0,1,2,3,4,5,6,7,8], ['WT 1', 'E64V', 'R73G', 'M100W', 'E64V/\nR73G','E64V/\nM100W', 'WT 2', 'D155S', 'D155S/\nM100W'], fontsize=18)
plt.ylabel('Variant', fontsize=22)
ax.get_yticklabels()[2].set_color("grey")
ax.get_yticklabels()[3].set_color("grey")
ax.get_yticklabels()[8].set_color("grey")

plt.plot([31.6], [2], '<', 'k', ms=12, color='grey')
plt.plot([31.6], [3], '<', 'k', ms=12, color='grey')
plt.plot([31.6], [8], '<', 'k', ms=12, color='grey')


plt.xlim(31,58)
plt.tight_layout()
plt.savefig('./figures/Tm_E64V_mutants.tiff', format='tiff', dpi=600)

In [ ]:
stats.ttest_ind([38.0, 37.9, 38], [55.8, 55.8, 55.8], equal_var=False)

In [ ]:
stats.ttest_ind([53.1, 53.8, 53.1], [55.8, 55.8, 55.8], equal_var=False)

In [ ]:
stats.ttest_ind([53.9, 54, 53.7], [55.8, 55.8, 55.8], equal_var=False)

In [ ]:
stats.ttest_ind([53.9, 54, 53.7], [53.1, 53.8, 53.1], equal_var=False)

In [ ]:
enzyme_assay_1 = pd.read_excel('./spectra/01112023_activity_test_format.xlsx', na_values='Overflow')
enzyme_assay_1

In [ ]:
enzyme_assay_1.columns

In [ ]:
enzyme_assay_1_sample_dict = {1:'WT',
                              2:'E64V',
                              3:'R73G',
                              4:'M100W',
                              5:'E64V-R73G', 
                              6:'E64V-M100W',
                              7:'blank'}

In [ ]:
enzyme_assay_1['sample'] = enzyme_assay_1.apply((lambda x: enzyme_assay_1_sample_dict[int(x['well'][-1])]), axis=1)


enzyme_assay_1

In [ ]:
enzyme_assay_2 = pd.read_excel('./spectra/02112023_activity_test_format.xlsx', na_values='Overflow')
enzyme_assay_2

In [ ]:
enzyme_assay_1_blank_1 = enzyme_assay_1[enzyme_assay_1['well']=='A7']
enzyme_assay_1_blank_2 = enzyme_assay_1[enzyme_assay_1['well']=='B7']
enzyme_assay_1_blank_3 = enzyme_assay_1[enzyme_assay_1['well']=='C7']


In [ ]:
enzyme_assay_1_WT_1 = enzyme_assay_1[enzyme_assay_1['well']=='A1']
enzyme_assay_1_WT_2 = enzyme_assay_1[enzyme_assay_1['well']=='B1']
enzyme_assay_1_WT_3 = enzyme_assay_1[enzyme_assay_1['well']=='C1']

In [ ]:
enzyme_assay_1_WT_1

In [ ]:
plt.plot(enzyme_assay_1_WT_1['Time [s]'], enzyme_assay_1_WT_1[237])
plt.plot(enzyme_assay_1_WT_2['Time [s]'], enzyme_assay_1_WT_2[237])
plt.plot(enzyme_assay_1_WT_3['Time [s]'], enzyme_assay_1_WT_3[237])


plt.plot(enzyme_assay_1_blank_1['Time [s]'], enzyme_assay_1_blank_1[237])
plt.plot(enzyme_assay_1_blank_2['Time [s]'], enzyme_assay_1_blank_2[237])
plt.plot(enzyme_assay_1_blank_3['Time [s]'], enzyme_assay_1_blank_3[237])

plt.axhline(3.06)
plt.axhline(1.17)

In [ ]:
stats.linregress(enzyme_assay_1_blank_1['Time [s]'], enzyme_assay_1_blank_1[237])

In [ ]:
stats.linregress(enzyme_assay_1_blank_2['Time [s]'], enzyme_assay_1_blank_2[237])

In [ ]:
stats.linregress(enzyme_assay_1_blank_3['Time [s]'], enzyme_assay_1_blank_3[237])

In [ ]:
(-1.945830288948559e-06*11609)+3.0910696272332303

In [ ]:
blank_mean = (enzyme_assay_1_blank_1[237].to_numpy()+enzyme_assay_1_blank_2[237].to_numpy()+enzyme_assay_1_blank_3[237].to_numpy())/3

In [ ]:
stats.linregress(enzyme_assay_1_blank_3['Time [s]'], blank_mean)

In [ ]:
[1,2]+[2,3]

In [ ]:
max_abs =  list(enzyme_assay_1_blank_1[237].to_numpy().flatten())+list(enzyme_assay_1_blank_2[237].to_numpy().flatten())+list(enzyme_assay_1_blank_3[237].to_numpy().flatten())

np.mean(max_abs)

In [ ]:
min_abs = (enzyme_assay_1_WT_1.at[129, 237]+enzyme_assay_1_WT_2.at[259, 237]+enzyme_assay_1_WT_3.at[389, 237])/3
min_abs

In [ ]:
enzyme_assay_1_WT_3

In [ ]:
enzyme_assay_1['FU_converted'] = enzyme_assay_1.apply((lambda x: (np.mean(max_abs)-x[237])/(np.mean(max_abs)-min_abs)),axis=1)

In [ ]:
enzyme_assay_2_sample_dict = {1:'WT',
                              2:'R53I',
                              3:'D155S',
                              4:'R53I-M100W',
                              5:'D155S-M100W', 
                              6:'NA',
                              7:'blank'}

In [ ]:
enzyme_assay_2_blank_1 = enzyme_assay_2[enzyme_assay_2['well']=='A7']
enzyme_assay_2_blank_2 = enzyme_assay_2[enzyme_assay_2['well']=='B7']
enzyme_assay_2_blank_3 = enzyme_assay_2[enzyme_assay_2['well']=='C7']


In [ ]:
enzyme_assay_2_WT_1 = enzyme_assay_2[enzyme_assay_2['well']=='A1']
enzyme_assay_2_WT_2 = enzyme_assay_2[enzyme_assay_2['well']=='B1']
enzyme_assay_2_WT_3 = enzyme_assay_2[enzyme_assay_2['well']=='C1']

In [ ]:
blank_mean = (enzyme_assay_2_blank_1[237].to_numpy()+enzyme_assay_2_blank_2[237].to_numpy()+enzyme_assay_2_blank_3[237].to_numpy())/3

In [ ]:
stats.linregress(enzyme_assay_2_blank_3['Time [s]'], blank_mean)

In [ ]:
max_abs =  list(enzyme_assay_2_blank_1[237].to_numpy().flatten())+list(enzyme_assay_2_blank_2[237].to_numpy().flatten())+list(enzyme_assay_2_blank_3[237].to_numpy().flatten())

np.mean(max_abs)

In [ ]:
min_abs = (enzyme_assay_2_WT_1.at[129, 237]+enzyme_assay_2_WT_2.at[259, 237]+enzyme_assay_2_WT_3.at[389, 237])/3
min_abs

In [ ]:
enzyme_assay_2['FU_converted'] = enzyme_assay_2.apply((lambda x: (np.mean(max_abs)-x[237])/(np.mean(max_abs)-min_abs)),axis=1)

enzyme_assay_2['sample'] = enzyme_assay_2.apply((lambda x: enzyme_assay_2_sample_dict[int(x['well'][-1])]), axis=1)


In [ ]:
time_vals_1 = list(enzyme_assay_1['Time [s]'])

rel_to_wt_assay_1 = {}

for time in time_vals_1:
    mean_FU_converted = np.mean(enzyme_assay_1[(enzyme_assay_1['sample']=='WT')&(enzyme_assay_1['Time [s]']==time)]['FU_converted'])
    rel_to_wt_assay_1[time]=mean_FU_converted

time_vals_2 = list(enzyme_assay_2['Time [s]'])   
rel_to_wt_assay_2 = {}

for time in time_vals_2:
    mean_FU_converted = np.mean(enzyme_assay_2[(enzyme_assay_2['sample']=='WT')&(enzyme_assay_2['Time [s]']==time)]['FU_converted'])
    rel_to_wt_assay_2[time]=mean_FU_converted

#rel_to_wt_assay_2

In [ ]:
enzyme_assay_1['rel_to_WT'] = enzyme_assay_1.apply(lambda x: x.FU_converted/(rel_to_wt_assay_1[x['Time [s]']]), axis=1)
enzyme_assay_2['rel_to_WT'] = enzyme_assay_2.apply(lambda x: x.FU_converted/(rel_to_wt_assay_2[x['Time [s]']]), axis=1)

enzyme_assay_1

In [ ]:
color_dict = {'WT':'black',
              'R53I':sns.color_palette('Greys', 10)[9],
              'E64V':sns.color_palette('Greys', 10)[7],
              'D155S':sns.color_palette('Greys', 10)[5],
              
              'R73G':sns.color_palette('Blues', 10)[9],
              'M100W':sns.color_palette('Blues', 10)[6],
              
              'E64V-R73G': sns.color_palette('plasma', 10)[3],
              'E64V-M100W': sns.color_palette('plasma', 10)[0],
              'R53I-M100W': sns.color_palette('plasma', 10)[7],
              'D155S-M100W': sns.color_palette('plasma', 10)[9]}

In [ ]:
from matplotlib.lines import Line2D


In [ ]:
legend_elements = [Line2D([0], [0], color=sns.color_palette('Greys', 10)[9], label='R53I', linestyle='-', marker=None, lw=4),
                   Line2D([0], [0], color=sns.color_palette('Greys', 10)[7], label='E64V', linestyle='-', marker=None, lw=4),
                   Line2D([0], [0], color=sns.color_palette('Greys', 10)[5], label='D155S', linestyle='-', marker=None, lw=4),
                   
                   Line2D([0], [0], color=sns.color_palette('Blues', 10)[9], label='R73G', linestyle='-', marker=None, lw=4),
                   Line2D([0], [0], color=sns.color_palette('Blues', 10)[6], label='M100W', linestyle='-', marker=None, lw=4)]
                   


In [ ]:
plt.figure(figsize=(4.5,4.5), facecolor='white')
sns.lineplot(data=enzyme_assay_1[(enzyme_assay_1['Time [s]']>0)&(enzyme_assay_1['sample'].isin(['E64V', 'R73G', 'M100W']))], 
             x='Time [s]', y='rel_to_WT', hue='sample', palette=color_dict, lw=4, alpha=0.9)

sns.lineplot(data=enzyme_assay_2[(enzyme_assay_2['Time [s]']>0)&(enzyme_assay_2['sample'].isin(['R53I', 'D155S']))],
             x='Time [s]', y='rel_to_WT', hue='sample', palette=color_dict, lw=4, alpha=0.9)

plt.legend(handles=legend_elements, fontsize=14, loc=(0.1,0.3), title_fontsize=14, frameon=False, ncol=2)



plt.xlim(-100, 11609)
plt.ylim(-0.05,1.05)
plt.xlabel('Time (s)', fontsize=19)
plt.ylabel('5-FC converted relative to WT', fontsize=19)
plt.xticks(fontsize=13)
plt.yticks(fontsize=13)
plt.title('Single variants', fontsize=19)

plt.savefig('./figures/fig4_a_1.svg', format='svg', dpi=600)

In [ ]:
legend_elements = [                  
                   Line2D([0], [0], color=sns.color_palette('plasma', 10)[3], label='E64V/R73G', linestyle='-', marker=None, lw=4),
                   Line2D([0], [0], color=sns.color_palette('plasma', 10)[0], label='E64V/M100W', linestyle='-', marker=None, lw=4),
                   Line2D([0], [0], color=sns.color_palette('plasma', 10)[7], label='R53I/M100W', linestyle='-', marker=None, lw=4),
                   Line2D([0], [0], color=sns.color_palette('plasma', 10)[9], label='D155S/M100W', linestyle='-', marker=None, lw=4)]

In [ ]:
plt.figure(figsize=(4.5,4.5), facecolor='white')
sns.lineplot(data=enzyme_assay_1[(enzyme_assay_1['Time [s]']>0)&(enzyme_assay_1['sample'].isin(['E64V-R73G', 'E64V-M100W']))], 
             x='Time [s]', y='rel_to_WT', hue='sample', palette=color_dict, lw=4, alpha=0.9)

sns.lineplot(data=enzyme_assay_2[(enzyme_assay_2['Time [s]']>0)&(enzyme_assay_2['sample'].isin(['R53I-M100W', 'D155S-M100W']))],
             x='Time [s]', y='rel_to_WT', hue='sample', palette=color_dict, lw=4, alpha=0.9)

plt.legend(handles=legend_elements, fontsize=14, loc=(0.38,0.115), title_fontsize=14, frameon=False, ncol=1)



plt.xlim(-100, 11609)
plt.ylim(-0.05,1.05)
plt.xlabel('Time (s)', fontsize=19)
plt.ylabel('5-FC converted relative to WT', fontsize=19)
plt.xticks(fontsize=13)
plt.yticks(fontsize=13)
plt.title('Co-expressed', fontsize=19)

plt.savefig('./figures/fig4_a_2.svg', format='svg', dpi=600)

## Protein stability measurements

In [ ]:
ti_df = pd.read_excel('./spectra/heteromer_assay_cleaved.xlsx')
ti_df

In [ ]:
plt.figure(figsize=(3.5,4.5), facecolor='white')

ax = sns.swarmplot(data=ti_df, 
                   order = ['WT_1', 'WT_2', 'E64V', 'D155S', 'R73G', 'M100W', 'E64V/R73G', 'E64V/M100W', 'D155S/M100W'],
                   y='variant', x='Tm', dodge=True, size=8, orient='h', color='k', alpha=0.75)

sample_list = ['WT_1', 'WT_2', 'E64V', 'D155S', 'R73G', 'M100W', 'E64V/R73G', 'E64V/M100W', 'D155S/M100W']

for y in range(0,9):
    median = np.mean(ti_df[ti_df['variant']==sample_list[y]]['Tm'])
    plt.plot([median, median], [y-0.2, y+0.2], '-', lw=3, zorder=9, color='red')
    #print(sample_list[y], stats.t.interval(alpha=0.95, df=3-1, loc=mean, scale=sem))
stats.t.interval(alpha=0.95, df=3-1, loc=37.98, scale=0.05)


plt.xlabel('T$_{m}$  ($^\circ$C)', fontsize=19)
plt.xticks(fontsize=16)

plt.yticks([0,1,2,3,4,5,6,7,8], ['WT$_{1}$', 'WT$_{2}$', 'E64V', 'D155S', 'R73G', 'M100W', 'E64V/R73G', 'E64V/M100W', 'D155S/M100W'], fontsize=12)
plt.ylabel('Variant', fontsize=19, labelpad=-20)
ax.get_yticklabels()[4].set_color("grey")

plt.plot([38.5], [4], '<', 'k', ms=12, color='grey')


plt.xlim(38, 62)
plt.ylim(9,-1)

plt.tight_layout()

plt.savefig('./figures/fig4_c.svg', format='svg', dpi=600)

In [ ]:
stats.ttest_ind(ti_df[ti_df['variant']=='E64V'].Tm, ti_df[(ti_df['variant']=='WT_1')|(ti_df['variant']=='WT_2')].Tm, equal_var=False)

In [ ]:
stats.ttest_ind(ti_df[ti_df['variant']=='M100W'].Tm, ti_df[(ti_df['variant']=='WT_1')|(ti_df['variant']=='WT_2')].Tm, equal_var=False)

In [ ]:
stats.ttest_ind(ti_df[ti_df['variant']=='D155S'].Tm, ti_df[(ti_df['variant']=='WT_1')|(ti_df['variant']=='WT_2')].Tm, equal_var=False)

## KD data

In [ ]:
test_monolith_dict = {1:{'time':0, 'Kd':724, 'lower':73, 'upper':7192, 'signal/noise':4.1},
                     2:{'time':9/60, 'Kd':102, 'lower':22, 'upper':480, 'signal/noise':4.7},
                     3:{'time':22/60, 'Kd':26.6, 'lower':8.3, 'upper':85.2, 'signal/noise':6.8},
                     4:{'time':32/60, 'Kd':14.8, 'lower':5.1, 'upper':43.2, 'signal/noise':8.1},
                     5:{'time':70/60, 'Kd':5.83, 'lower':2.21, 'upper':15.41, 'signal/noise':11.6},
                     6:{'time':98/60, 'Kd':4.85, 'lower':2.01, 'upper':11.69, 'signal/noise':13.7},
                     7:{'time':132/60, 'Kd':4.82, 'lower':2.15, 'upper':10.74, 'signal/noise':15}}

data_monolith = pd.DataFrame.from_dict(test_monolith_dict, orient='index')

In [ ]:
plt.figure(figsize=(16,6), facecolor='white')
plt.subplot(121)

plt.plot(data_monolith['time'],data_monolith['Kd'], marker='o',  lw=3, ms=8, color='k')

plt.text(1.75, 15, '4.82 nM', fontsize=18)
plt.xlim(-0.025,2.4)

plt.yscale('log')

plt.fill_between(data_monolith['time'], data_monolith['lower'], data_monolith['upper'], color='lightgrey')

plt.xlabel('Time (hours)', fontsize=20)
plt.ylabel('FCY1$_{WT}$-FCY1$_{WT}$ K$_{D}$ (nM)', fontsize=20)

plt.xticks(fontsize=15)
plt.yticks(fontsize=15)



plt.subplot(122)
plt.plot(data_monolith['time'],data_monolith['signal/noise'], marker='o', lw=3, ms=8, color='k')
plt.ylim(0,16)
plt.xlabel('Time (hours)', fontsize=20)
plt.ylabel('Signal to noise ratio', fontsize=20)
plt.axhline(5, linestyle='--', color='red')
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)

plt.text(-3.45,15.8,'a', fontsize=28)
plt.text(-0.50,15.8,'b', fontsize=28)


plt.savefig('./figures/fig_KD.svg', format='svg', dpi=600)